In [1]:
import { Ollama } from "npm:langchain/llms/ollama";
import { AgentType } from "npm:langchain/agents";
import { RetrievalQAChain } from "npm:langchain/chains";
import { MemoryVectorStore } from "npm:langchain/vectorstores/memory";
import { OllamaEmbeddings } from "npm:langchain/embeddings/ollama";
import { RecursiveCharacterTextSplitter } from "npm:langchain/text_splitter";
import "npm:d3-dsv";

[Module: null prototype] {
  autoType: [Function: autoType],
  csvFormat: [Function: format],
  csvFormatBody: [Function: formatBody],
  csvFormatRow: [Function: formatRow],
  csvFormatRows: [Function: formatRows],
  csvFormatValue: [Function: formatValue],
  csvParse: [Function: parse],
  csvParseRows: [Function: parseRows],
  dsvFormat: [Function: default],
  tsvFormat: [Function: format],
  tsvFormatBody: [Function: formatBody],
  tsvFormatRow: [Function: formatRow],
  tsvFormatRows: [Function: formatRows],
  tsvFormatValue: [Function: formatValue],
  tsvParse: [Function: parse],
  tsvParseRows: [Function: parseRows]
}

In [2]:
// loaders
import { DirectoryLoader } from "npm:langchain/document_loaders/fs/directory"
import { JSONLoader } from "npm:langchain/document_loaders/fs/json"
import { TextLoader } from "npm:langchain/document_loaders/fs/text"
import { CSVLoader } from "npm:langchain/document_loaders/fs/csv"
import { PDFLoader } from "npm:langchain/document_loaders/fs/pdf"
import { Document } from "npm:langchain/document"

In [3]:
const loader = new DirectoryLoader("./docs", {
    ".json": (path) => new JSONLoader(path),
    ".txt": (path) => new TextLoader(path),
    ".csv": (path) => new CSVLoader(path, { separator: ","}),
    ".pdf": (path) => new PDFLoader(path),
});

In [4]:
const docs = await loader.load();
console.log(docs)

[
  Document {
    pageContent: "id: 1\n" +
      "first_name: Oli\n" +
      "last_name: Mound\n" +
      "email: omound@test.com\n" +
      "gender: Female\n" +
      "review: 3",
    metadata: { source: "/Users/jorishermans/docs/MOCK_DATA.csv", line: 1 }
  },
  Document {
    pageContent: "id: 2\n" +
      "first_name: Prince\n" +
      "last_name: Y\n" +
      "email: prince.y@abc.net.au\n" +
      "gender: Male\n" +
      "review: 8",
    metadata: { source: "/Users/jorishermans/docs/MOCK_DATA.csv", line: 2 }
  },
  Document {
    pageContent: "id: 3\n" +
      "first_name: Ricky\n" +
      "last_name: Martin\n" +
      "email: ricky.martin@glu.com\n" +
      "gender: Male\n" +
      "review: 3",
    metadata: { source: "/Users/jorishermans/docs/MOCK_DATA.csv", line: 3 }
  },
  Document {
    pageContent: "id: 4\n" +
      "first_name: Melanie\n" +
      "last_name: Pidgeon\n" +
      "email: mpidgeon3@test.com\n" +
      "gender: Female\n" +
      "review: 1",
    metadata: { sou

In [5]:
const csvContent = docs.map((doc: Document) => doc.pageContent);
console.log(csvContent);

[
  "id: 1\n" +
    "first_name: Oli\n" +
    "last_name: Mound\n" +
    "email: omound@test.com\n" +
    "gender: Female\n" +
    "review: 3",
  "id: 2\n" +
    "first_name: Prince\n" +
    "last_name: Y\n" +
    "email: prince.y@abc.net.au\n" +
    "gender: Male\n" +
    "review: 8",
  "id: 3\n" +
    "first_name: Ricky\n" +
    "last_name: Martin\n" +
    "email: ricky.martin@glu.com\n" +
    "gender: Male\n" +
    "review: 3",
  "id: 4\n" +
    "first_name: Melanie\n" +
    "last_name: Pidgeon\n" +
    "email: mpidgeon3@test.com\n" +
    "gender: Female\n" +
    "review: 1",
  "id: 5\n" +
    "first_name: Matthew\n" +
    "last_name: McWire\n" +
    "email: mmcwire@d.org\n" +
    "gender: Male\n" +
    "review: 1",
  "id: 6\n" +
    "first_name: Christine\n" +
    "last_name: Tyer\n" +
    "email: ctyer5@diigo.com\n" +
    "gender: Non-binary\n" +
    "review: 2",
  "id: 7\n" +
    "first_name: Adriaens\n" +
    "last_name: Anna\n" +
    "email: aa@virginia.edu\n" +
    "gender: Fe

In [6]:
const askModel = async (question: string) => {
    const model = new Ollama({ model: "mixtral" });
    let vectorstore;

    const textSplitter = new RecursiveCharacterTextSplitter({
      chunkSize: 500,
      chunkOverlap: 450
    });

    console.log("Text Splitting......");
	console.log(`Chunk size  ----> ${textSplitter.chunkSize}`);
	console.log(`Chunk Overlap  ----> ${textSplitter.chunkOverlap}`);

	const splitDocs = await textSplitter.createDocuments(csvContent);
    console.log(splitDocs)

	const vectorStore = await MemoryVectorStore.fromDocuments(
		splitDocs,
		new OllamaEmbeddings({ model: "mixtral" })
	);

    // RetrievalQAChain
	const chain = RetrievalQAChain.fromLLM(model, vectorStore.asRetriever());
	console.log("Querying...");
	const res = await chain.call({ query: question });
	console.log(res);
    return res;
}

In [ ]:
const res = await askModel(`We provided you with data around reviews. 
                           Your task is to determine how many reviews we have knowing that the maximum score is 10, 
                           starting from 0 up to number 10 ?
                           Give me your detailed reasoning why you came to that conclusion.`);
console.log(res.text);

Text Splitting......
Chunk size  ----> 500
Chunk Overlap  ----> 450
[
  Document {
    pageContent: "id: 1\n" +
      "first_name: Oli\n" +
      "last_name: Mound\n" +
      "email: omound@test.com\n" +
      "gender: Female\n" +
      "review: 3",
    metadata: { loc: { lines: { from: 1, to: 6 } } }
  },
  Document {
    pageContent: "id: 2\n" +
      "first_name: Prince\n" +
      "last_name: Y\n" +
      "email: prince.y@abc.net.au\n" +
      "gender: Male\n" +
      "review: 8",
    metadata: { loc: { lines: { from: 1, to: 6 } } }
  },
  Document {
    pageContent: "id: 3\n" +
      "first_name: Ricky\n" +
      "last_name: Martin\n" +
      "email: ricky.martin@glu.com\n" +
      "gender: Male\n" +
      "review: 3",
    metadata: { loc: { lines: { from: 1, to: 6 } } }
  },
  Document {
    pageContent: "id: 4\n" +
      "first_name: Melanie\n" +
      "last_name: Pidgeon\n" +
      "email: mpidgeon3@test.com\n" +
      "gender: Female\n" +
      "review: 1",
    metadata: { loc:

In [ ]:
const res = await askModel("Can you give me the exact dataset that you have printed out in columns?");
console.log(res.text);

In [ ]:
const res = await askModel("What are the different persons that provided a review of 3? Provide the anwser in short format.");
console.log(res.text);

In [ ]:
const res = await askModel("What are the different persons that provided a review score of 3 in the column review? Give me the reason why you pick these persons?");
console.log(res.text);

In [ ]:
const res = await askModel("What are the genders in that filled in the reviews ?");
console.log(res.text);

In [ ]:
const res = await askModel("How provided the highest score in the column review?");
console.log(res.text);

In [ ]:
const res = await askModel("How provided the highest score in the column review? Know that the highest score is 10.");
console.log(res.text);